In [1]:
import pandas as pd
import numpy as np
import os
import ast
import matplotlib.pyplot as plt
from huggingface_hub import hf_hub_download
import seaborn as sns
from collections import Counter
import warnings
import fasttext
import umap
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize

tokens_by_class = {
    "TYPE": set(),
    "BRAND": set(),
    "VOLUME": set(),
    "PERCENT": set()
}

In [2]:
# Настройки для визуализаций
sns.set_theme(style="whitegrid", palette="viridis")
plt.rcParams["figure.figsize"] = (12, 7)
warnings.filterwarnings("ignore")

# Константы для путей к файлам
DATA_RAW_PATH = "../../data/raw/"
TRAIN_FILE = DATA_RAW_PATH + "train.csv"
SUBMISSION_FILE = DATA_RAW_PATH + "submission.csv"
EDA_PATH = "artifacts/"
os.makedirs(EDA_PATH, exist_ok=True)

In [3]:
df_train = pd.read_csv(TRAIN_FILE, sep=";")
df_submission = pd.read_csv(SUBMISSION_FILE, sep=";")

print("Обучающий набор данных (train.csv):")
df_train.info()
print("\n")

print("\n" + "=" * 50 + "\n")

print("Тестовый набор данных (submission.csv):")
df_submission.info()
print("\n")

Обучающий набор данных (train.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27251 entries, 0 to 27250
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sample      27251 non-null  object
 1   annotation  27251 non-null  object
dtypes: object(2)
memory usage: 425.9+ KB




Тестовый набор данных (submission.csv):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   sample      5000 non-null   object
 1   annotation  5000 non-null   object
dtypes: object(2)
memory usage: 78.3+ KB




In [4]:
print("Тип данных колонки 'annotation' до преобразования:", df_train["annotation"].dtype)

# Применяем безопасный парсинг
df_train["annotation"] = df_train["annotation"].apply(ast.literal_eval)

print("Тип данных колонки 'annotation' после преобразования:", df_train["annotation"].dtype)
print("\nПример данных в колонке 'annotation' после преобразования:")
print(df_train["annotation"].iloc[5])

Тип данных колонки 'annotation' до преобразования: object
Тип данных колонки 'annotation' после преобразования: object

Пример данных в колонке 'annotation' после преобразования:
[(0, 6, 'B-BRAND'), (7, 12, 'B-TYPE')]


Анализ длины запросов

In [5]:
df_train["query_len_char"] = df_train["sample"].str.len()
df_train["query_len_token"] = df_train["sample"].str.split().str.len()

df_submission["query_len_char"] = df_submission["sample"].str.len()
df_submission["query_len_token"] = df_submission["sample"].str.split().str.len()

print("Статистика по длинам запросов в обучающем наборе:")
display(df_train[["query_len_char", "query_len_token"]].describe())

print("\nСтатистика по длинам запросов в тестовом наборе:")
display(df_submission[["query_len_char", "query_len_token"]].describe())


plt.figure(figsize=(14, 6))
sns.histplot(df_train["query_len_char"], color="blue", label="Train", kde=True, stat="density", linewidth=0)
sns.histplot(df_submission["query_len_char"], color="red", label="Submission", kde=True, stat="density", linewidth=0, alpha=0.6)
plt.title("Распределение длин запросов в символах (Train vs Submission)")
plt.xlabel("Длина запроса в символах")
plt.ylabel("Плотность")
plt.legend()
plt.savefig(os.path.join(EDA_PATH, "Распределение длин запросов в символах (Train vs Submission).png"), dpi=300, bbox_inches="tight")
plt.close()

plt.figure(figsize=(14, 6))
sns.histplot(df_train["query_len_token"], color="blue", label="Train", kde=True, stat="density", linewidth=0, binwidth=1)
sns.histplot(df_submission["query_len_token"], color="red", label="Submission", kde=True, stat="density", linewidth=0, alpha=0.6, binwidth=1)
plt.title("Распределение длин запросов в токенах (Train vs Submission)")
plt.xlabel("Длина запроса в токенах")
plt.ylabel("Плотность")
plt.legend()
plt.xlim(0, 20)
plt.xticks(range(0, 21))
plt.savefig(os.path.join(EDA_PATH, "Распределение длин запросов в токенах (Train vs Submission).png"), dpi=300, bbox_inches="tight")
plt.close()

Статистика по длинам запросов в обучающем наборе:


,query_len_char,query_len_token
count,27251.00000,27251.000000
mean,10.36762,1.550512
std,4.97992,0.714341
min,2.00000,1.000000
25%,7.00000,1.000000
50%,9.00000,1.000000
75%,14.00000,2.000000
max,45.00000,6.000000



Статистика по длинам запросов в тестовом наборе:


,query_len_char,query_len_token
count,5000.000000,5000.000000
mean,10.409400,1.553000
std,5.003679,0.723392
min,2.000000,1.000000
25%,7.000000,1.000000
50%,9.000000,1.000000
75%,14.000000,2.000000
max,39.000000,5.000000


Анализ мультиязычности

In [6]:
def contains_latin(text):
    if not isinstance(text, str):
        return False
    return any("a" <= char.lower() <= "z" for char in text)

In [7]:
df_train["contains_latin"] = df_train["sample"].apply(contains_latin)
df_submission["contains_latin"] = df_submission["sample"].apply(contains_latin)

train_latin_ratio = df_train["contains_latin"].mean()
submission_latin_ratio = df_submission["contains_latin"].mean()

latin_ratio_df = pd.DataFrame({
    "Набор": ["train", "test"],
    "Доля_латиницы": [train_latin_ratio, submission_latin_ratio]
})
latin_ratio_df.to_csv(os.path.join(EDA_PATH, "latin_ratio.csv"), index=False)

df_train[df_train["contains_latin"]].sample(10, random_state=42).to_csv(
    os.path.join(EDA_PATH, "train_latin_samples.csv"), index=False
)

print(f"Доля запросов с латиницей в обучающем наборе: {train_latin_ratio:.2%}")
print(f"Доля запросов с латиницей в тестовом наборе: {submission_latin_ratio:.2%}")

print("\nПримеры запросов с латиницей из обучающего набора:")
display(df_train[df_train["contains_latin"]].sample(10, random_state=42))

Доля запросов с латиницей в обучающем наборе: 14.48%
Доля запросов с латиницей в тестовом наборе: 14.52%

Примеры запросов с латиницей из обучающего набора:


,sample,annotation,query_len_char,query_len_token,contains_latin
6731,дрожжи nord,"[(0, 6, B-TYPE), (7, 11, B-BRAND)]",11,2,True
1110,j7￼,"[(0, 3, B-BRAND)]",3,1,True
3773,бренди torre,"[(0, 6, B-TYPE), (7, 12, B-BRAND)]",12,2,True
70,altero,"[(0, 6, B-BRAND)]",6,1,True
1517,milkyve,"[(0, 7, B-BRAND)]",7,1,True
6205,груша papadimitri,"[(0, 5, B-TYPE), (6, 17, B-BRAND)]",17,2,True
1242,l.g,"[(0, 3, O)]",3,1,True
229,beker,"[(0, 5, B-BRAND)]",5,1,True
764,exxe порошк,"[(0, 4, B-BRAND), (5, 11, B-TYPE)]",11,2,True
366,bushido,"[(0, 7, B-BRAND)]",7,1,True


Анализ баланса классов сущностей

In [8]:
def get_entity_type(tag):
    if "-" in tag:
        return tag.split("-")[1]
    return tag

In [9]:
entity_counter = Counter()
for annotation_list in df_train["annotation"]:
    for _, _, tag in annotation_list:
        entity_type = get_entity_type(tag)
        entity_counter[entity_type] += 1

df_entity_counts = pd.DataFrame(entity_counter.items(), columns=["Entity", "Count"]).sort_values("Count", ascending=False)

df_entity_counts.to_csv(os.path.join(EDA_PATH, "entity_counts.csv"))

print("Распределение сущностей в обучающем наборе:")
display(df_entity_counts)


plt.figure(figsize=(10, 6))
sns.barplot(x="Entity", y="Count", data=df_entity_counts)
plt.title("Распределение количества сущностей по типам")
plt.xlabel("Тип сущности")
plt.ylabel("Количество")
for index, row in df_entity_counts.iterrows():
    plt.text(row.name, row.Count, row.Count, color="black", ha="center", va="bottom")
plt.savefig(os.path.join(EDA_PATH, "Распределение количества сущностей по типам.png"), dpi=300, bbox_inches="tight")
plt.close()

Распределение сущностей в обучающем наборе:


,Entity,Count
2,TYPE,29060
1,BRAND,7699
0,O,5379
5,VOLUME,84
3,PERCENT,30
4,0,1


Частотный анализ

In [10]:
all_tokens = df_train["sample"].dropna().str.lower().str.split().sum()

token_counts = Counter(all_tokens)

top_tokens_df = pd.DataFrame(token_counts.most_common(30), columns=["Токен", "Частота"])
top_tokens_df.to_csv(os.path.join(EDA_PATH, "top_tokens.csv"), index=False)

print("Топ-30 самых частотных слов в обучающем наборе:")
display(pd.DataFrame(token_counts.most_common(30), columns=["Токен", "Частота"]))

Топ-30 самых частотных слов в обучающем наборе:


,Токен,Частота
0,для,796
1,с,465
2,сыр,271
3,в,219
4,хлеб,159
5,сок,134
6,вода,134
7,корм,128
8,без,117
9,чай,116


Анализ неоднозначности токенов

In [11]:
token_to_tags = {}

for _, row in df_train.iterrows():
    query = row["sample"]
    annotations = row["annotation"]
    
    for start, end, tag in annotations:
        token = query[start:end].lower()
        entity_type = get_entity_type(tag) 
        
        if token not in token_to_tags:
            token_to_tags[token] = set()
        token_to_tags[token].add(entity_type)

ambiguous_tokens = {token: tags for token, tags in token_to_tags.items() if len(tags) > 1}

df_ambiguous = pd.DataFrame(ambiguous_tokens.items(), columns=["Токен", "Возможные теги"])

df_ambiguous.head(20).reset_index(drop=True).to_csv(
    os.path.join(EDA_PATH, "ambiguous_head20.csv"), index=False
)

print(f"Найдено {len(df_ambiguous)} неоднозначных токенов (могут иметь разные теги в разных контекстах).")
print("\nПримеры 20 самых частотных неоднозначных токенов:")

df_ambiguous["Частота"] = df_ambiguous["Токен"].apply(lambda x: token_counts.get(x, 0))
df_ambiguous = df_ambiguous.sort_values("Частота", ascending=False)

display(df_ambiguous.head(20).reset_index(drop=True))

Найдено 852 неоднозначных токенов (могут иметь разные теги в разных контекстах).

Примеры 20 самых частотных неоднозначных токенов:


,Токен,Возможные теги,Частота
0,для,"{TYPE, O}",796
1,с,"{TYPE, O, BRAND}",465
2,сыр,"{TYPE, O, BRAND}",271
3,в,"{TYPE, O, BRAND}",219
4,хлеб,"{TYPE, BRAND}",159
5,вода,"{TYPE, O}",134
6,сок,"{TYPE, O}",134
7,корм,"{TYPE, O}",128
8,без,"{TYPE, O}",117
9,чай,"{TYPE, O}",116


Анализ контекстного окружения сущностей

In [12]:
# Ячейка N: Анализ контекстного окружения сущностей

from collections import Counter
from IPython.display import display

# --- Добавляем эту вспомогательную функцию ---
def get_entity_type(tag: str) -> str:
    """Извлекает "чистый" тип сущности из BIO-тега."""
    if tag == "O":
        return "O"
    if "-" in tag:
        return tag.split("-")[1]
    return tag # На случай, если тег уже "чистый"

# Инициализируем словари для хранения контекста
valid_entities = {"TYPE", "BRAND", "VOLUME", "PERCENT"}
context_before = {entity: Counter() for entity in valid_entities}
context_after = {entity: Counter() for entity in valid_entities}

# Используем наш отлаженный датафрейм df_train, где аннотации уже распарсены
for _, row in df_train.iterrows():
    # Используем простой split() для анализа, так как нам важны слова, а не сабворды
    query_tokens = str(row["sample"]).lower().split()

    # Создаем карту позиций для токенов
    current_pos = 0
    token_indices = []
    text_lower = str(row["sample"]).lower()
    for token in query_tokens:
        try:
            start = text_lower.index(token, current_pos)
            end = start + len(token)
            token_indices.append((start, end))
            current_pos = end
        except ValueError:
            # Если токен не найден (из-за нормализации или спецсимволов), пропускаем
            token_indices.append((-1, -1))

    # Проходим по аннотациям
    for start_ann, end_ann, tag_ann in row["annotation"]:
        entity_type = get_entity_type(tag_ann)

        # --- КЛЮЧЕВОЕ ИСПРАВЛЕНИЕ ---
        # Проверяем, что извлеченный тип сущности валиден
        if entity_type not in valid_entities:
            continue

        # Находим токены, соответствующие аннотации
        for i, (start_tok, end_tok) in enumerate(token_indices):
            if start_tok == -1: continue # Пропускаем ненайденные токены

            if max(start_ann, start_tok) < min(end_ann, end_tok):
                # Собираем контекст
                if i > 0:
                    prev_token = query_tokens[i-1]
                    context_before[entity_type][prev_token] += 1

                if i < len(query_tokens) - 1:
                    next_token = query_tokens[i+1]
                    context_after[entity_type][next_token] += 1

# Выводим результаты
for entity_type in valid_entities:
    print(f"\n===== Контекст для сущности: {entity_type} =====")

    df_before = pd.DataFrame(context_before[entity_type].most_common(10), columns=["Слово до", "Частота"])
    df_after = pd.DataFrame(context_after[entity_type].most_common(10), columns=["Слово после", "Частота"])

    os.makedirs(EDA_PATH, exist_ok=True)

    df_before.to_csv(os.path.join(EDA_PATH, f"context_before_{entity_type}.csv"), sep=";", index=False)
    df_after.to_csv(os.path.join(EDA_PATH, f"context_after_{entity_type}.csv"), sep=";", index=False)

    print("Топ-10 слов, встречающихся ДО сущности:")
    display(df_before)

    print("\nТоп-10 слов, встречающихся ПОСЛЕ сущности:")
    display(df_after)


===== Контекст для сущности: TYPE =====
Топ-10 слов, встречающихся ДО сущности:


,Слово до,Частота
0,сыр,197
1,хлеб,93
2,прочие,76
3,колбаса,64
4,чай,58
5,масло,53
6,сок,50
7,вода,50
8,корм,48
9,artfruit,40



Топ-10 слов, встречающихся ПОСЛЕ сущности:


,Слово после,Частота
0,для,754
1,с,431
2,в,201
3,без,101
4,доя,70
5,из,59
6,красная,49
7,по,45
8,на,42
9,сыр,33



===== Контекст для сущности: PERCENT =====
Топ-10 слов, встречающихся ДО сущности:


...

,Слово после,Частота
0,л,9
1,литра,4
2,кг,4
3,литров,3
4,литр,2
5,штук,2
6,детская,1
7,красная,1
8,объём,1
9,global,1



===== Контекст для сущности: BRAND =====
Топ-10 слов, встречающихся ДО сущности:


,Слово до,Частота
0,сок,37
1,красная,33
2,вода,32
3,фруто,31
4,сады,21
5,чай,19
6,конфеты,19
7,молоко,19
8,горошек,18
9,кофейник,17



Топ-10 слов, встречающихся ПОСЛЕ сущности:


,Слово после,Частота
0,цена,33
1,няня,31
2,например,10
3,и,10
4,придонья,10
5,в,9
6,польза,9
7,может,9
8,дома,8
9,без,8


Анализ пространства признаков

In [14]:
for _, row in df_train.iterrows():
    query = row["sample"]
    annotations = row["annotation"]
    
    for start, end, tag in annotations:
        entity_type = get_entity_type(tag)
        if entity_type in tokens_by_class:
            token = query[start:end].lower()
            tokens_by_class[entity_type].add(token)

all_unique_tokens = []
labels = []
for entity_type, token_set in tokens_by_class.items():
    tokens_list = list(token_set)
    all_unique_tokens.extend(tokens_list)
    labels.extend([entity_type] * len(tokens_list))

print("Подготовка данных завершена.")
for entity_type, token_set in tokens_by_class.items():
    print(f"Найдено {len(token_set)} уникальных токенов для класса '{entity_type}'")

Подготовка данных завершена.
Найдено 15639 уникальных токенов для класса 'TYPE'
Найдено 4206 уникальных токенов для класса 'BRAND'
Найдено 36 уникальных токенов для класса 'VOLUME'
Найдено 19 уникальных токенов для класса 'PERCENT'


In [15]:
print("Загрузка модели FastText (cc.ru.300.bin) из Hugging Face Hub...")
print("Это может занять значительное время при первом запуске.")

try:
    model_path = hf_hub_download(repo_id="facebook/fasttext-ru-vectors", filename="model.bin")

    ft_model = fasttext.load_model(model_path)
    print("Модель FastText успешно загружена.")

except Exception as e:
    print(f"Произошла ошибка при загрузке модели FastText: {e}")
    print("Убедитесь, что вы аутентифицированы в Hugging Face и есть стабильное интернет-соединение.")
    ft_model = None

if ft_model:
    print("Извлечение векторов для уникальных токенов...")
    ft_embeddings = np.array([ft_model.get_word_vector(token) for token in all_unique_tokens])

    print("Применение UMAP для понижения размерности эмбеддингов...")
    reducer_ft = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42)
    embedding_2d_ft = reducer_ft.fit_transform(ft_embeddings)
    print("Понижение размерности завершено.")

    print("Создание и сохранение графика визуализации...")
    plt.figure(figsize=(14, 12))

    df_plot_ft = pd.DataFrame({"x": embedding_2d_ft[:, 0], "y": embedding_2d_ft[:, 1], "label": labels})

    scatter = sns.scatterplot(data=df_plot_ft, x="x", y="y", hue="label", s=25, alpha=0.8, palette="viridis")

    plt.title("Визуализация Эмбеддингов Токенов (FastText + UMAP)", fontsize=16)
    plt.xlabel("UMAP Компонента 1", fontsize=12)
    plt.ylabel("UMAP Компонента 2", fontsize=12)
    plt.legend(title="Тип сущности", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.grid(True, linestyle="--", alpha=0.6)

    output_image_path = os.path.join(EDA_PATH, "fasttext_umap_visualization.png")
    plt.savefig(output_image_path, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"График успешно сохранен по пути: {output_image_path}")

Загрузка модели FastText (cc.ru.300.bin) из Hugging Face Hub...
Это может занять значительное время при первом запуске.


model.bin:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

Модель FastText успешно загружена.
Извлечение векторов для уникальных токенов...
Применение UMAP для понижения размерности эмбеддингов...
Понижение размерности завершено.
Создание и сохранение графика визуализации...
График успешно сохранен по пути: artifacts/fasttext_umap_visualization.png


Векторизация TF-IDF и n-gramms

In [16]:
print("Векторизация с помощью TF-IDF на n-граммах символов...")
tfidf_vectorizer = TfidfVectorizer(analyzer="char", ngram_range=(2, 4))
tfidf_embeddings = tfidf_vectorizer.fit_transform(all_unique_tokens)

tfidf_embeddings_normalized = normalize(tfidf_embeddings)


print("Применение UMAP для TF-IDF эмбеддингов...")
reducer_tfidf = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42, metric="hellinger")
embedding_2d_tfidf = reducer_tfidf.fit_transform(tfidf_embeddings_normalized)

plt.figure(figsize=(12, 10))
df_plot_tfidf = pd.DataFrame({"x": embedding_2d_tfidf[:, 0], "y": embedding_2d_tfidf[:, 1], "label": labels})
sns.scatterplot(data=df_plot_tfidf, x="x", y="y", hue="label", s=20, alpha=0.7)
plt.title("Визуализация токенов (TF-IDF на n-граммах символов + UMAP)")
plt.xlabel("UMAP компонента 1")
plt.ylabel("UMAP компонента 2")
plt.legend(title="Тип сущности")
plt.savefig(os.path.join(EDA_PATH, "Визуализация токенов (TF-IDF на n-граммах символов + UMAP).png"), dpi=300, bbox_inches="tight")
plt.close()

Векторизация с помощью TF-IDF на n-граммах символов...
Применение UMAP для TF-IDF эмбеддингов...
